# Azure AI Agent service - MCP support

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol

Supported regions: westus, westus2, uaenorth, southindia and switzerlandnorth

Install required packages:

`pip install gradio python-dotenv azure-identity azure-ai-projects>=1.0.0b12 azure-ai-agents>=1.1.0b4 --pre`

In [10]:
import os
import gradio as gr
import json
import re
import sys
import time

from azure.ai.agents.models import ListSortOrder, MessageTextContent, McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [11]:
sys.version

'3.12.1 (main, May 30 2025, 12:50:35) [GCC 13.3.0]'

## 1. Settings

In [12]:
#load_dotenv("azure.env")
load_dotenv()

PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT2")

In [13]:
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT,
                                 credential=DefaultAzureCredential())

# Convenience handle for the Agents sub‑client
agents_client = project_client.agents

In [14]:
#model = "gpt-4o"
model = "gpt-4.1"

## 2. Example

In [15]:
# With this prefix 'https://gitmcp.io/' you can instantly create a remote RCP server from any GitHub repo
github_url = "https://github.com/Azure-Samples/ai-foundry-agents-samples"

get_substr = re.search(r'\.com/(.*)', github_url)
substr = get_substr.group(1)
mcp_server_url = f"https://gitmcp.io/{substr}"

mcp_server_url

'https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples'

In [16]:
prompt = "Any python code for a mcp server with Azure AI Agent?"

In [ ]:
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT,
                                 credential=DefaultAzureCredential())

# MCP tool definition
mcp_tool = McpTool(
    server_label="github",
    server_url=mcp_server_url
    #allowed_tools=[],          # Optional initial allow‑list
)

# Agent creation
agent = agents_client.create_agent(
    model=model,
    name="agent-mcp",
    instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
    tools=mcp_tool.definitions
)
print(f"🎉 Created agent, agent ID: {agent.id}")
print(f"✨ MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Thread creation
thread = agents_client.threads.create()
print(f"🧵 Created thread, thread ID: {thread.id}")

# Message creation
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"💬 Created message, message ID: {message.id}")

mcp_tool.set_approval_mode("never")          # Disable human approval

# Run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(2)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"⏳ Run status: {run.status}")
if run.status == "failed":
    print(f"❌ Run error: {run.last_error}")

# Run steps
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
print()

for step in run_steps:
    print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"🛠️ Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=5))


HttpResponseError: (Tenant provided in token does not match resource token) Token tenant 72f988bf-86f1-41af-91ab-2d7cd011db47 does not match resource tenant.
Code: Tenant provided in token does not match resource token
Message: Token tenant 72f988bf-86f1-41af-91ab-2d7cd011db47 does not match resource tenant.

In [ ]:
messages = agents_client.messages.list(thread_id=thread.id,
                                               order=ListSortOrder.ASCENDING)

for item in messages:
    last_message_content = item.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{item.role}: {last_message_content.text.value}")

user: Any python code for a mcp server with Azure AI Agent?
assistant: For implementing a Python-based MCP (Model Control Protocol) server integrated with Azure AI Agents, the repository "Azure-Samples/ai-foundry-agents-samples" on GitHub provides concrete examples. One specific example under the folder `examples/mcp/streamable-http` contains the sample code for:

1. **MCP Server**: Using Streamable HTTP transport for interactions.
2. **MCP Client**: Implemented using the Semantic Kernel Python SDK.
3. **AI Foundry Agent**: An AI service MCP client integration.

### Repository Contents
The example provides three main folders:
- `mcp-server`: Contains the Python sample code for setting up an MCP server using Streamable HTTP transport.
- `mcp-client`: Python MCP Client implementation connected with Azure Semantic Kernel functionalities.
- `ai-foundry-agent`: Connection to Azure AI Foundry Agent services.

### Steps to Explore Code:
1. Navigate to the **[MCP Streamable HTTP Example](https

## Another example

In [10]:
# With this prefix 'https://gitmcp.io/' you can instantly create a remote RCP server from any GitHub repo
github_url = "https://github.com/openai/openai-python"

get_substr = re.search(r'\.com/(.*)', github_url)
substr = get_substr.group(1)
mcp_server_url = f"https://gitmcp.io/{substr}"

mcp_server_url

'https://gitmcp.io/openai/openai-python'

In [11]:
prompt = "Show me some python code to use whisper"

In [ ]:
# Agent creation
agent = agents_client.create_agent(
    model=model,
    name="agent-mcp",
    instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
    tools=[{
        "type": "mcp",  # mcp
        "server_label": "github",  # label
        "server_url": mcp_server_url,  # url
        "require_approval": "never"  # no approval
    }],
    tool_resources=None
)
print(f"🎉 Created agent, agent ID: {agent.id}")

# Thread creation
thread = agents_client.threads.create()
print(f"🧵 Created thread, thread ID: {thread.id}")

# Message creation
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"💬 Created message, message ID: {message.id}")

# Run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(2)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"⏳ Run status: {run.status}")
if run.status == "failed":
    print(f"❌ Run error: {run.last_error}")

# Run steps
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
print()

for step in run_steps:
    print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"🛠️ Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=5))


🎉 Created agent, agent ID: asst_pZWKMLeQt8vi79pjxDTdYCmJ
🧵 Created thread, thread ID: thread_RtwptyR6Ucuprl7WQgXTuFJf
💬 Created message, message ID: msg_qy64RVRNn7TXcCU9civtrPlv
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: completed

🔄 Run step: step_SEOTK89PmeB9PDHGhdRHr5qA, status: completed, type: message_creation
🔄 Run step: step_qeij2g9CzR5puVLRdjnax9ys, status: completed, type: tool_calls
🛠️ Tool call details:
{
     "id": "call_vK5GKQAvtmgjVCKsHoSNZze6",
     "type": "mcp",
     "arguments": "{\"url\":\"https://github.com/openai/openai-python/blob/1dbb72b290cb360a9ccbcec17425aaba4ad114b0/examples/audio.py\"}",
     "name": "fetch_generic_url_content",
     "output": "[Skip to content](#start-of-content)

In [ ]:
messages = agents_client.messages.list(thread_id=thread.id,
                                               order=ListSortOrder.ASCENDING)

for item in messages:
    last_message_content = item.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{item.role}: {last_message_content.text.value}")

user: Show me some python code to use whisper
assistant: Here is a Python code snippet that demonstrates how to use OpenAI's Whisper model for transcription, translation, and generating text-to-speech audio using the `openai-python` library. This code was sourced from their `audio.py` example file.

### Example Python Code:
```python
#!/usr/bin/env python

from pathlib import Path
from openai import OpenAI

# Initialize OpenAI client (API Key will be inferred from environment variables)
openai = OpenAI()

# Path for saving audio file
speech_file_path = Path(__file__).parent / "speech.mp3"

def main():
    # Create text-to-speech (audio) file
    with openai.audio.speech.with_streaming_response.create(
        model="tts-1",  # Text-to-speech model
        voice="alloy", # Customize voice details
        input="The quick brown fox jumped over the lazy dogs",
    ) as response:
        response.stream_to_file(speech_file_path)

    # Perform transcription of an audio file
    transcripti

## 3. Webapp

In [ ]:
def agent_mcp_fn(github_url, prompt):
    """
    Creates an AI agent to interact with a specified GitHub repository and processes a given prompt.

    Args:
        github_url (str): The URL of the GitHub repository.
        prompt (str): The prompt to be processed by the agent.

    Returns:
        tuple: A tuple containing:
            - str: The concatenated responses from the agent.
            - str: The URLs extracted from the agent's responses.

    Raises:
        ValueError: If the GitHub URL format is invalid.
        RuntimeError: If the agent's run fails.
        Exception: For any other errors that occur during execution.
    """
    try:
        get_substr = re.search(r'\.com/(.*)', github_url)

        if get_substr:
            substr = get_substr.group(1)
        else:
            raise ValueError("Invalid URL format.")
        
        mcp_server_url = f"https://gitmcp.io/{substr}"

        # Agent creation
        agent = agents_client.create_agent(
            model=model,
            name="agent-mcp",
            instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
            tools=[{
                "type": "mcp",  # mcp
                "server_label": "github",  # label
                "server_url": mcp_server_url,  # url
                "require_approval": "never"  # no approval
            }],
            tool_resources=None
        )
        print(f"🎉 Created agent, agent ID: {agent.id}")

        # Thread creation
        thread = agents_client.threads.create()
        print(f"🧵 Created thread, thread ID: {thread.id}")

        # Message creation
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=prompt,
        )
        print(f"💬 Created message, message ID: {message.id}")

        # Run
        run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(2)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
            print(f"⏳ Run status: {run.status}")
        if run.status == "failed":
            raise RuntimeError(f"❌ Run error: {run.last_error}")

        # Run steps
        run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
        
        for step in run_steps:
            print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
            if step.type == "tool_calls":
                print(f"🛠️ Tool call details:")
                for tool_call in step.step_details.tool_calls:
                    print(json.dumps(tool_call.as_dict(), indent=5))

        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

        answers = []
        for item in messages:
            last_message_content = item.content[-1]
            if isinstance(last_message_content, MessageTextContent):
                answers.append(f"{item.role}: {last_message_content.text.value}")
        
        url_template = r'(https?://[^\s]+|www\.[^\s]+)'
        urls = re.findall(url_template, "\n".join(answers))
        url_links = ' '.join(urls).replace(')', '').replace(' ', '\n')

        return "\n".join(answers), url_links
        
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [15]:
def build_webapp():
    with gr.Blocks(title="Azure AI Foundry Agent") as demo:
        gr.Markdown("# 🤖 Azure AI Foundry Agent")
        gr.Markdown("## Enter a GitHub repository URL and your prompt to interact with the MCP server using Azure AI agent.")

        with gr.Row():
            github_url = gr.Textbox(label="🔗 GitHub Repository URL", placeholder="e.g. https://github.com/org/repo")
            prompt = gr.Textbox(label="📝 Prompt", placeholder="What does this GitHub repo do?", lines=2)

        with gr.Row():
            results_output = gr.Textbox(label="📄 Agent response", lines=25, interactive=False)
            links_output = gr.Textbox(label="🔍 Links", lines=5, interactive=False)

        run_button = gr.Button("🚀 Run Agent")
        run_button.click(fn=agent_mcp_fn, inputs=[github_url, prompt], outputs=[results_output, links_output])

    return demo


webapp = build_webapp()
webapp.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://bb3a90ccf464d0bdc3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎉 Created agent, agent ID: asst_bSoDTpa1cGnzRnSirKYqSLMO
🧵 Created thread, thread ID: thread_7bOcOYkvqjNETWQOLztJ6rYd
💬 Created message, message ID: msg_DeBan9nDoIILGpHsAzNsOL6L
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: in_progress
⏳ Run status: completed
🔄 Run step: step_Ugwuaafy7VpfGJ4T5FenAkXr, status: completed, type: message_creation
🔄 Run step: step_GxXTv5qjHV4YtOK7eaT3Ffjy, status: completed, type: tool_calls
🛠️ Tool call details:
{
     "id": "call_N8I83LfGvtBV2V92STJki